<a href="https://colab.research.google.com/github/PacktPublishing/Hands-On-Computer-Vision-with-PyTorch/blob/master/Chapter08/Training_YOLO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import platform
curr_system = platform.system()
print(curr_system)
# brew install opencv

Darwin


In [2]:
%%time
import os
if not os.path.exists('darknet'):
    !git clone https://github.com/AlexeyAB/darknet
    %cd darknet    
    if curr_system == 'Darwin':
        !gsed -i 's/OPENCV=0/OPENCV=1/' Makefile
    else:
        !sed -i 's/OPENCV=0/OPENCV=1/' Makefile
    # !!! In case you dont have a GPU, make sure to comment out the below 3 lines !!! #
#     !sed -i 's/GPU=0/GPU=1/' Makefile
#     !sed -i 's/CUDNN=0/CUDNN=1/' Makefile
#     !sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
    !make
    !pip install -q torch_snippets
    !wget --quiet https://www.dropbox.com/s/agmzwk95v96ihic/open-images-bus-trucks.tar.xz
    !tar -xf open-images-bus-trucks.tar.xz
    !rm open-images-bus-trucks.tar.xz
    !wget --quiet https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights

Cloning into 'darknet'...
remote: Enumerating objects: 15502, done.
remote: Total 15502 (delta 0), reused 0 (delta 0), pack-reused 15502
Receiving objects: 100% (15502/15502), 14.15 MiB | 1.20 MiB/s, done.
Resolving deltas: 100% (10404/10404), done.
Updating files: 100% (2057/2057), done.
/Users/santhom/GithubProjects/Modern-Computer-Vision-with-PyTorch/chap08-advanced-object-detection/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp:94:13: warning: 'image_to_mat' has C-linkage specified, but returns user-defined type 'cv::Mat' which is incompatible with C [-Wreturn-type-c-linkage]
    cv::Mat image_to_mat(image img);
            ^
./src/image_opencv.cpp:136:9: warning: 'load_image_mat' has

./src/image_opencv.cpp:450:14: warning: unused function 'get_pixel' [-Wunused-function]
static float get_pixel(image m, int x, int y, int c)
             ^
17 warnings generated.
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -c ./src/http_stream.cpp -o obj/http_stream.o
./src/http_stream.cpp:253:21: warning: unused variable 'n' [-Wunused-variable]
                int n = _write(client, outputbuf, outlen);
                    ^
./src/http_stream.cpp:511:17: warning: 'sprintf' is deprecated: This function is provided for compatibility reasons only.  Due to security concerns inherent in the design of sprintf(3), it is highly recommended that you use snprintf(3) instead. [-Wdeprecated-declarations]
                sprintf(head, "--mjpegstream\r\nContent-Type: image/jpeg\r\nContent-Length: %zu\r\n\r\n", outlen);
  

./src/activations.c:79:12: warning: 7 enumeration values not handled in switch: 'RELU6', 'SWISH', 'MISH'... [-Wswitch]
    switch(a){
           ^
./src/activations.c:79:12: note: add missing switch cases
    switch(a){
           ^
./src/activations.c:310:12: warning: enumeration values 'SWISH', 'MISH', and 'HARD_MISH' not handled in switch [-Wswitch]
    switch(a){
           ^
./src/activations.c:310:12: note: add missing switch cases
    switch(a){
           ^
2 warnings generated.
gcc -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -c ./src/im2col.c -o obj/im2col.o
gcc -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -c ./src/col2im.c -o obj/col2im.o
gcc -Iinclude/ -I3rdparty/stb/incl

2 warnings generated.
gcc -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -c ./src/matrix.c -o obj/matrix.o
gcc -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -c ./src/network.c -o obj/network.o
./src/network.c:435:13: warning: unused variable 'ema_period' [-Wunused-variable]
        int ema_period = (net.max_batches - ema_start_point - 1000) * (1.0 - net.ema_alpha);
            ^
./src/network.c:1269:14: warning: unused function 'relu' [-Wunused-function]
static float relu(float src) {
             ^
2 warnings generated.
gcc -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -Wall -Wfatal-errors -Wno-unused-result -Wno-unk

./src/detector.c:588:26: warning: expression result unused [-Wunused-value]
                    for (k; buf[k + n] != '\0'; k++)
                         ^
./src/detector.c:707:13: warning: unused variable 'mkd' [-Wunused-variable]
        int mkd = make_directory(buff, 0777);
            ^
./src/detector.c:709:13: warning: unused variable 'mkd2' [-Wunused-variable]
        int mkd2 = make_directory(buff2, 0777);
            ^
./src/detector.c:1326:24: warning: variable 'cur_prob' set but not used [-Wunused-but-set-variable]
                double cur_prob = 0;
                       ^
./src/detector.c:1346:15: warning: unused variable 'class_precision' [-Wunused-variable]
        float class_precision = (float)tp_for_thresh_per_class[i] / ((float)tp_for_thresh_per_class[i] + (float)fp_for_thresh_per_class[i]);
              ^
./src/detector.c:1347:15: warning: unused variable 'class_recall' [-Wunused-variable]
        float class_recall = (float)tp_for_thresh_per_class[i] / ((float)tp

1 warning generated.
gcc -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -c ./src/cifar.c -o obj/cifar.o
./src/cifar.c:32:180: warning: format specifies type 'long' but the argument has type 'uint64_t' (aka 'unsigned long long') [-Wformat]
        printf("%d, %.3f: %f, %f avg, %f rate, %lf seconds, %ld images\n", get_current_batch(net), (float)(*net.seen)/N, loss, avg_loss, get_current_rate(net), sec(clock()-time), *net.seen);
                                                            ~~~                                                                                                                    ^~~~~~~~~
                                                            %llu
./src/cifar.c:88:180: warning: format specifies type 'long' but the argument has type 'uint64_t' (aka 'unsigned long long') [-Wformat]
        printf("%d, %.3f: %f,

20 warnings generated.
gcc -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -c ./src/gaussian_yolo_layer.c -o obj/gaussian_yolo_layer.o
gcc -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -c ./src/upsample_layer.c -o obj/upsample_layer.o
gcc -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -c ./src/lstm_layer.c -o obj/lstm_layer.o
gcc -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -c ./src/conv

In [3]:
%%time 
#33.1s
!./darknet detector test cfg/coco.data cfg/yolov4.cfg yolov4.weights data/person.jpg

 GPU isn't used 
 OpenCV version: 4.7.0
mini_batch = 1, batch = 8, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 1    608 x 608 x   3 ->  608 x 608 x  32 0.639 BF
   1 conv     64       3 x 3/ 2    608 x 608 x  32 ->  304 x 304 x  64 3.407 BF
   2 conv     64       1 x 1/ 1    304 x 304 x  64 ->  304 x 304 x  64 0.757 BF
   3 route  1 		                           ->  304 x 304 x  64 
   4 conv     64       1 x 1/ 1    304 x 304 x  64 ->  304 x 304 x  64 0.757 BF
   5 conv     32       1 x 1/ 1    304 x 304 x  64 ->  304 x 304 x  32 0.379 BF
   6 conv     64       3 x 3/ 1    304 x 304 x  32 ->  304 x 304 x  64 3.407 BF
   7 Shortcut Layer: 4,  wt = 0, wn = 0, outputs: 304 x 304 x  64 0.006 BF
   8 conv     64       1 x 1/ 1    304 x 304 x  64 ->  304 x 304 x  64 0.757 BF
   9 route  8 2 	                           ->  304 x 304 x 128 
  10 conv     64       1 x 1/ 1    304 x 304 x 128 ->  304 x 304 x  64 1.5

 104 conv   1024       1 x 1/ 1     19 x  19 x1024 ->   19 x  19 x1024 0.757 BF
 105 conv    512       1 x 1/ 1     19 x  19 x1024 ->   19 x  19 x 512 0.379 BF
 106 conv   1024       3 x 3/ 1     19 x  19 x 512 ->   19 x  19 x1024 3.407 BF
 107 conv    512       1 x 1/ 1     19 x  19 x1024 ->   19 x  19 x 512 0.379 BF
 108 max                5x 5/ 1     19 x  19 x 512 ->   19 x  19 x 512 0.005 BF
 109 route  107 		                           ->   19 x  19 x 512 
 110 max                9x 9/ 1     19 x  19 x 512 ->   19 x  19 x 512 0.015 BF
 111 route  107 		                           ->   19 x  19 x 512 
 112 max               13x13/ 1     19 x  19 x 512 ->   19 x  19 x 512 0.031 BF
 113 route  112 110 108 107 	                   ->   19 x  19 x2048 
 114 conv    512       1 x 1/ 1     19 x  19 x2048 ->   19 x  19 x 512 0.757 BF
 115 conv   1024       3 x 3/ 1     19 x  19 x 512 ->   19 x  19 x1024 3.407 BF
 116 conv    512       1 x 1/ 1     19 x  19 x1024 ->   19 x  19 x 512 0.379 BF

^C
CPU times: user 1min 24s, sys: 24.7 s, total: 1min 49s
Wall time: 1h 25min 38s


In [7]:
%%writefile data/obj.names
bus
truck

Writing data/obj.names


In [9]:
%%writefile data/obj.data
classes = 2
train = data/train.txt
valid = data/val.txt
names = data/obj.names
backup = backup/

Writing data/obj.data


In [10]:
%%time 
#52s
!mkdir -p data/obj
!cp -r open-images-bus-trucks/images/* data/obj/
!cp -r open-images-bus-trucks/yolo_labels/all/{train,val}.txt data/
!cp -r open-images-bus-trucks/yolo_labels/all/labels/*.txt data/obj/

/usr/local/bin/bash: line 1: /bin/cp: Argument list too long
CPU times: user 599 ms, sys: 185 ms, total: 784 ms
Wall time: 35.9 s


In [11]:
%%time
# create a copy of existing configuration and modify it in place
!cp cfg/yolov4-tiny-custom.cfg cfg/yolov4-tiny-bus-trucks.cfg
# max_batches to 4000 (since the dataset is small enough)

# Note SED commands on Mac require gsed, since default is BSD sed
# modifying commands to use gsed (has to be installed locally through Homebrew)

if curr_system == 'Darwin':
    !gsed -i 's/max_batches = 500200/max_batches=4000/' cfg/yolov4-tiny-bus-trucks.cfg
    # number of sub-batches per batch
    !gsed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov4-tiny-bus-trucks.cfg
    # number of batches after which learning rate is decayed
    !gsed -i 's/steps=400000,450000/steps=3200,3600/' cfg/yolov4-tiny-bus-trucks.cfg
    # number of classes is 2 as opposed to 80 (which is the number of COCO classes)
    !gsed -i 's/classes=80/classes=2/g' cfg/yolov4-tiny-bus-trucks.cfg
    # in the classification and regression heads, change number of output convolution filters
    # from 255 -> 21 and 57 -> 33, since we have fewer classes we don't need as many filters
    !gsed -i 's/filters=255/filters=21/g' cfg/yolov4-tiny-bus-trucks.cfg
    !gsed -i 's/filters=57/filters=33/g' cfg/yolov4-tiny-bus-trucks.cfg
else:    
    !sed -i 's/max_batches = 500200/max_batches=4000/' cfg/yolov4-tiny-bus-trucks.cfg
    # number of sub-batches per batch
    !sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov4-tiny-bus-trucks.cfg
    # number of batches after which learning rate is decayed
    !sed -i 's/steps=400000,450000/steps=3200,3600/' cfg/yolov4-tiny-bus-trucks.cfg
    # number of classes is 2 as opposed to 80 (which is the number of COCO classes)
    !sed -i 's/classes=80/classes=2/g' cfg/yolov4-tiny-bus-trucks.cfg
    # in the classification and regression heads, change number of output convolution filters
    # from 255 -> 21 and 57 -> 33, since we have fewer classes we don't need as many filters
    !sed -i 's/filters=255/filters=21/g' cfg/yolov4-tiny-bus-trucks.cfg
    !sed -i 's/filters=57/filters=33/g' cfg/yolov4-tiny-bus-trucks.cfg

CPU times: user 16.6 ms, sys: 43.7 ms, total: 60.4 ms
Wall time: 978 ms


In [12]:
%%time
!wget --quiet https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.conv.29
!cp yolov4-tiny.conv.29 build/darknet/x64/

CPU times: user 40.8 ms, sys: 25.4 ms, total: 66.2 ms
Wall time: 1.86 s


In [ ]:
%%time
!./darknet detector train data/obj.data cfg/yolov4-tiny-bus-trucks.cfg yolov4-tiny.conv.29 -dont_show -mapLastAt

 GPU isn't used 
 OpenCV version: 4.7.0
yolov4-tiny-bus-trucks
mini_batch = 4, batch = 64, time_steps = 1, train = 1 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 2    416 x 416 x   3 ->  208 x 208 x  32 0.075 BF
   1 conv     64       3 x 3/ 2    208 x 208 x  32 ->  104 x 104 x  64 0.399 BF
   2 conv     64       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x  64 0.797 BF
   3 route  2 		                       1/2 ->  104 x 104 x  32 
   4 conv     32       3 x 3/ 1    104 x 104 x  32 ->  104 x 104 x  32 0.199 BF
   5 conv     32       3 x 3/ 1    104 x 104 x  32 ->  104 x 104 x  32 0.199 BF
   6 route  5 4 	                           ->  104 x 104 x  64 
   7 conv     64       1 x 1/ 1    104 x 104 x  64 ->  104 x 104 x  64 0.089 BF
   8 route  2 7 	                           ->  104 x 104 x 128 
   9 max                2x 2/ 2    104 x 104 x 128 ->   52 x  52 x 128 0.001 BF
  10 conv    128       3 x 3/ 1     52 x  52 x 128 ->   52 x

Can't open label file. (This can be normal only if you use MSCOCO): data/obj/abd096be4cb545b5.txt 
Can't open label file. (This can be normal only if you use MSCOCO): data/obj/450f6c7adb38f9d4.txt 
Can't open label file. (This can be normal only if you use MSCOCO): data/obj/0364c26ec2861b9a.txt 
Can't open label file. (This can be normal only if you use MSCOCO): data/obj/a8d3fc6847ba6593.txt 
Can't open label file. (This can be normal only if you use MSCOCO): data/obj/004e26b41de45ed0.txt 
Can't open label file. (This can be normal only if you use MSCOCO): data/obj/06a9ced593c9a4f6.txt 
Can't open label file. (This can be normal only if you use MSCOCO): data/obj/c03534004b4fab5a.txt 
Can't open label file. (This can be normal only if you use MSCOCO): data/obj/18f0111fc20fa2a8.txt 
Can't open label file. (This can be normal only if you use MSCOCO): data/obj/fe9cad85a35abca5.txt 
Can't open label file. (This can be normal only if you use MSCOCO): data/obj/7b6d950d1231569c.txt 
Can't open

v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 37 Avg (IOU: 0.000000), count: 1, class_loss = 668.737183, iou_loss = 0.000000, total_loss = 668.737183 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 30 Avg (IOU: 0.000000), count: 1, class_loss = 182.786514, iou_loss = 0.000000, total_loss = 182.786514 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 37 Avg (IOU: 0.000000), count: 1, class_loss = 665.036621, iou_loss = 0.000000, total_loss = 665.036621 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 30 Avg (IOU: 0.000000), count: 1, class_loss = 181.084656, iou_loss = 0.000000, total_loss = 181.084656 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 37 Avg (IOU: 0.000000), count: 1, class_loss = 661.442017, iou_loss = 0.000000, total_loss = 661.442017 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 30 Avg (IOU: 0.000000), count: 1, class_loss = 179.356506, iou_loss = 0.0

Can't open label file. (This can be normal only if you use MSCOCO): data/obj/5a3d7b3c593c92e5.txt 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 30 Avg (IOU: 0.000000), count: 1, class_loss = 180.897705, iou_loss = 0.000000, total_loss = 180.897705 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 37 Avg (IOU: 0.000000), count: 1, class_loss = 665.692810, iou_loss = 0.000000, total_loss = 665.692810 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 30 Avg (IOU: 0.000000), count: 1, class_loss = 180.461624, iou_loss = 0.000000, total_loss = 180.461624 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 37 Avg (IOU: 0.000000), count: 1, class_loss = 662.724854, iou_loss = 0.000000, total_loss = 662.724854 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 30 Avg (IOU: 0.000000), count: 1, class_loss = 180.787689, iou_loss = 0.000000, total_loss = 180.787689 
v3 (iou loss, Normalizer: (iou: 0.07, obj

Can't open label file. (This can be normal only if you use MSCOCO): data/obj/b6b2a94b6339d4c5.txt 
Can't open label file. (This can be normal only if you use MSCOCO): data/obj/20fad61e874a8bb8.txt 
Can't open label file. (This can be normal only if you use MSCOCO): data/obj/0b7d3757f721051c.txt 
Can't open label file. (This can be normal only if you use MSCOCO): data/obj/90224ec7b921a2da.txt 
Can't open label file. (This can be normal only if you use MSCOCO): data/obj/80085fe24ac52561.txt 
Can't open label file. (This can be normal only if you use MSCOCO): data/obj/0b4b368709476b67.txt 
Can't open label file. (This can be normal only if you use MSCOCO): data/obj/29cb3ec68c90621a.txt 
Can't open label file. (This can be normal only if you use MSCOCO): data/obj/e7a7113487d423b0.txt 
Can't open label file. (This can be normal only if you use MSCOCO): data/obj/c75b8881c1fcd250.txt 
Can't open label file. (This can be normal only if you use MSCOCO): data/obj/ed83d397cbddbdc9.txt 
Can't open

...

Can't open label file. (This can be normal only if you use MSCOCO): data/obj/18a3cebf0ea70540.txt 
Can't open label file. (This can be normal only if you use MSCOCO): data/obj/ec01f35a924d09ad.txt 
Can't open label file. (This can be normal only if you use MSCOCO): data/obj/9e8eb8e92112becd.txt 
Can't open label file. (This can be normal only if you use MSCOCO): data/obj/dabca34e416af7f5.txt 
Can't open label file. (This can be normal only if you use MSCOCO): data/obj/bad221bc6a40f4e3.txt 
Can't open label file. (This can be normal only if you use MSCOCO): data/obj/193de4a2b77a4fc4.txt 
Can't open label file. (This can be normal only if you use MSCOCO): data/obj/193de4a2b77a4fc4.txt 
Can't open label file. (This can be normal only if you use MSCOCO): data/obj/36bbbacfdc9a2c70.txt 
Can't open label file. (This can be normal only if you use MSCOCO): data/obj/830aaae6cf14e06b.txt 
Can't open label file. (This can be normal only if you use MSCOCO): data/obj/83da3493a8fc1c2c.txt 
Can't open

Can't open label file. (This can be normal only if you use MSCOCO): data/obj/6de9b376b80165bb.txt 
Can't open label file. (This can be normal only if you use MSCOCO): data/obj/c1e7365228884e0e.txt 
Can't open label file. (This can be normal only if you use MSCOCO): data/obj/958c67b15dea2904.txt 
Can't open label file. (This can be normal only if you use MSCOCO): data/obj/fc104396d9f64897.txt 
Can't open label file. (This can be normal only if you use MSCOCO): data/obj/14660c6929f66455.txt 
Can't open label file. (This can be normal only if you use MSCOCO): data/obj/1a10471e7e6280e8.txt 
Can't open label file. (This can be normal only if you use MSCOCO): data/obj/66959c6006680764.txt 
Can't open label file. (This can be normal only if you use MSCOCO): data/obj/6f96ebc8fb7a3534.txt 
Can't open label file. (This can be normal only if you use MSCOCO): data/obj/bb13f76fe1f4e9bd.txt 
Can't open label file. (This can be normal only if you use MSCOCO): data/obj/a3a66c555a8f1328.txt 
Can't open

v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 30 Avg (IOU: 0.000000), count: 1, class_loss = 108.860718, iou_loss = 0.000000, total_loss = 108.860718 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 37 Avg (IOU: 0.000000), count: 1, class_loss = 431.853760, iou_loss = 0.000000, total_loss = 431.853760 
]2;95/4000: loss=270.4 hours left=196.7
 95: 270.429138, 316.358154 avg loss, 0.000000 rate, 173.368575 seconds, 6080 images, 196.651886 hours left
Loaded: 0.000044 seconds
Can't open label file. (This can be normal only if you use MSCOCO): data/obj/5769148e00951a3f.txt 
Can't open label file. (This can be normal only if you use MSCOCO): data/obj/7d16366cd221b8da.txt 
Can't open label file. (This can be normal only if you use MSCOCO): data/obj/ee97359ace342186.txt 
Can't open label file. (This can be normal only if you use MSCOCO): data/obj/d54899b9f1593b5e.txt 
Can't open label file. (This can be normal only if you use MSCOCO): data/obj/5f3da3a8

v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 30 Avg (IOU: 0.000000), count: 1, class_loss = 107.499657, iou_loss = 0.000000, total_loss = 107.499657 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 37 Avg (IOU: 0.000000), count: 1, class_loss = 418.257751, iou_loss = 0.000000, total_loss = 418.257751 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 30 Avg (IOU: 0.000000), count: 1, class_loss = 107.352989, iou_loss = 0.000000, total_loss = 107.352989 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 37 Avg (IOU: 0.000000), count: 1, class_loss = 418.899689, iou_loss = 0.000000, total_loss = 418.899689 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 30 Avg (IOU: 0.000000), count: 1, class_loss = 107.289658, iou_loss = 0.000000, total_loss = 107.289658 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 37 Avg (IOU: 0.000000), count: 1, class_loss = 418.361298, iou_loss = 0.0

Can't open label file. (This can be normal only if you use MSCOCO): data/obj/ce09681e3890f0df.txt 
Can't open label file. (This can be normal only if you use MSCOCO): data/obj/821519604bd489fd.txt 
Can't open label file. (This can be normal only if you use MSCOCO): data/obj/fbbad7f7aa174e7f.txt 
Can't open label file. (This can be normal only if you use MSCOCO): data/obj/75f307bbe1989eb3.txt 
Can't open label file. (This can be normal only if you use MSCOCO): data/obj/f4d42e7dd4136206.txt 
Can't open label file. (This can be normal only if you use MSCOCO): data/obj/3eabc31aa9612a87.txt 
Can't open label file. (This can be normal only if you use MSCOCO): data/obj/7aefd23c953cd20e.txt 
Can't open label file. (This can be normal only if you use MSCOCO): data/obj/fdd294cf71bab30e.txt 
Can't open label file. (This can be normal only if you use MSCOCO): data/obj/1b713df7d836a274.txt 
Can't open label file. (This can be normal only if you use MSCOCO): data/obj/249e2380d5b581ca.txt 
Can't open

In [ ]:
%%time
from torch_snippets import Glob, stem, show, read
# upload your own images to a folder
image_paths = Glob('images-of-trucks-and-busses')
for f in image_paths:
    !./darknet detector test data/obj.data cfg/yolov4-tiny-bus-trucks.cfg\
    backup/yolov4-tiny-bus-trucks_4000.weights {f}
    !mv predictions.jpg {stem(f)}_pred.jpg

for i in Glob('*_pred.jpg', silent=True):
    show(read(i, 1), sz=20)